In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install langchain
!pip install faiss-gpu
!pip install sentence_transformers
!pip install pypdf
!pip install langchain-experimental


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 2

In [2]:
from google.colab import drive  ## Mount Drive
from huggingface_hub import notebook_login
from torch import cuda, bfloat16
import transformers
from langchain.document_loaders import PyPDFLoader
import re
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
from langchain.vectorstores  import FAISS
import gc
from langchain.llms import HuggingFacePipeline
from transformers import BitsAndBytesConfig
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
drive.mount('/content/drive',force_remount= True)


Mounted at /content/drive


In [5]:
cd drive/MyDrive/SS&C

/content/drive/MyDrive/SS&C


In [6]:

notebook_login()

In [7]:
def loadmodel(model_name):
  model_id = model_name
  device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
  bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)
  model_config = transformers.AutoConfig.from_pretrained(
    model_id
)
  model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)
  model = model.eval()
  print(f"Model loaded on {device}")
  return model
model = loadmodel('mistralai/Mistral-7B-Instruct-v0.1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Model loaded on cuda:0


In [8]:
#Initialize Tokenizer, Padding side left as right padding leads to un-controlled generation
#Adding end of sequence and beginning of dequence tokens.
##Padding is done using end of sequence tokens

model_id = 'mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    padding_Side = "left",
    add_eos_token = True,
    add_bos_token = True,

)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
#Parameter Tuning for text generation
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    task='text-generation',
    do_sample=True,
 ##Increasing Temp. raises randomness is answers
    temperature=0.2,
    max_new_tokens=2048,
    repetition_penalty=1.1
)

In [10]:
def cleaned_text(text):
  text = re.sub(r' +', ' ', text)
  text = text.replace('\n', ' ')
  text = re.sub(r'\s*-\s*', '', text)
  return text

In [11]:
def load_clean(book):
  try:

    loader = PyPDFLoader(book)
    pages = loader.load_and_split() ##Split book through pages and multi level headings
    pages = pages[80:] ## Starting after the contents of the title page / Starting from Chapter 1, till the end of the book
    text = ' '.join([ page.page_content.replace('\t', ' ') for page in pages ])
    clean_text = cleaned_text(text)
    return clean_text

  except:
    return("PDF file incorrect / not found at required location")




In [12]:
clean_text =  load_clean('book.pdf')

In [13]:
##Load Embedding Model. The current model is being chosen as it is small and ranks Number 8 on the MTEB Leaderboard, which ranks LLMs based on their retrieval and embedding capability
model_name = "mixedbread-ai/mxbai-embed-large-v1"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [14]:
#Semantic Chunking over Recursive or Character or Markdown
#Medical documents often contain complex concepts, technical terminology, and context-dependent information.
#Splitting the text based solely on characters or markdown structures may result in chunks that lack semantic coherence, making it difficult for the language model to understand
#and reason about the content effectively.
# Semantic chunking, on the other hand, aims to create chunks that are semantically meaningful and preserve the contextual information, which is crucial
#for accurate retrieval and generation in the RAG pipeline.
##breakpoint threshold is set to percentile after various iterations of this pipeline where interquartile and stand dard deviation did not bring about good retrieval results

semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")

In [15]:
docs = semantic_chunker.create_documents([clean_text])

In [16]:
vectorstore = FAISS.from_documents(docs,embeddings)

In [17]:

gc.collect()

18

In [18]:
llm = HuggingFacePipeline(pipeline = generate_text)

In [19]:
db = vectorstore.as_retriever(search_kwargs={"k": 4})


In [ ]:
##Below I showcase three solutions for Identifying Out of Context answer. The first using semantic / contextual similarity the second is inspired from chain of verification prompt using LLMs.
## The third is contextual based (LLM) models. Through my testing I believe Tf-IDF and verification through LLMs yeilded the best results

In [ ]:
##Method 1: TFIDF Vector Search

In [20]:
prompt_template = """
### [INST] Instruction: Answer the question based on your knowledge of Internal Medicine. Here is context to help: :

{context}


### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [21]:
def calculate_tfidf_similarity(string1, string2):
    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the strings into TF-IDF vectors
    vectors = vectorizer.fit_transform([string1, string2])

    # Calculate the cosine similarity between the two vectors
    similarity = cosine_similarity(vectors[0], vectors[1])

    return similarity[0][0]



In [22]:
def QABot(question,llm_chain):

  rag_chain = (
 {"context": db, "question": RunnablePassthrough()}
    | llm_chain
)
  result = rag_chain.invoke(question)

  context = "".join(str(x) for x in result['context'])
  text = "".join(str(x) for x in result['text'])

  #print(result['context'])

  #print(context)
  #print(text)

  ##Verification OOC
  string1 = context
  string2 = text
  tfidf_similarity = calculate_tfidf_similarity(string1, string2)
  print(tfidf_similarity)
  if(tfidf_similarity > 0.5):
    print(text)
  else:
    #print(text)
    print("Question is out of context")


In [23]:
question = "What are the common TYPES OF BACK PAIN?"
QABot(question,llm_chain)

0.7099378491164212

Back pain can be classified into several categories depending on its location, severity, and duration. The most common types of back pain include:

1. Acute low back pain (ALBP): This type of pain occurs suddenly and lasts for a short period of time, usually less than three months. It is usually caused by a strain or sprain of the muscles or ligaments in the lower back.
2. Chronic low back pain (CLBP): This type of pain persists for more than three months and can last for years. It can be caused by various factors such as disc herniation, spinal stenosis, osteoarthritis, or rheumatoid arthritis.
3. Disc herniation: This occurs when the soft gel-like center (nucleus pulposus) of a spinal disc bulges or ruptures, putting pressure on surrounding nerves. This can cause pain, numbness, tingling, and weakness in the back and legs.
4. Osteoarthritis: This is a degenerative joint disease that affects the cartilage that cushions the ends of bones. It can cause pain, stiffnes

In [24]:
question = "Who is Elon Musk?"
QABot(question,llm_chain)

0.0
Question is out of context


In [ ]:
###Method 2 - Using a verification prompt to check if text and context are similar

In [32]:
prompt_template = """
### [INST] Instruction: Answer the question based on your knowledge of Internal Medicine. Here is context to help: :

{context}


### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=llm, prompt=prompt)


In [33]:
##Verification Prompt

prompt_template_verify = """
### [INST]

Text: {text}

Here is the Context:

{context}

[/INST]
 """

prompt_2 = PromptTemplate(
    input_variables=["text", "context"],
    template=prompt_template_verify,
)

# Create llm chain
llm_chain_2 = LLMChain(llm=llm, prompt=prompt_2)

In [34]:
def QABot(question,llm_chain,llm_chain_2):

  rag_chain = (
 {"context": db, "question": RunnablePassthrough()}
    | llm_chain
)
  result = rag_chain.invoke(question)

  context = result['context']
  text = result['text']

  ##Verification OOC
  rag_chain_2 = (
 {"context": lambda x: context, "text":lambda x: text}
    | llm_chain_2
)
  result2 = rag_chain_2.invoke("Can only the context provided be used to generate the text. Answer Yes or No. ")

  #type(result2['text'])
  #print(result2['text'])
  if('Yes' in result2['text']):
    print(text)
    print(context)

  else:

    print("Out of Context Question")
    print(text)
    print(result2['text'])
    print(result2['context'])



In [36]:
question = 'Who is Elon Musk?'

QABot(question,llm_chain,llm_chain_2)

Out of Context Question

Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and lead designer of SpaceX, co-founder, CEO, and product architect of Tesla Inc., co-founder of Neuralink, and founder of The Boring Company. Musk has also founded several other companies, including Zip2, X.com, PayPal, and SolarCity. He is known for his innovative ideas and entrepreneurial spirit in various fields such as space exploration, electric vehicles, renewable energy, and artificial intelligence.

Based on the provided text and context, it appears that Elon Musk is an individual with diverse interests and expertise in multiple fields. He is a businessman, industrial designer, and engineer who has founded and led several successful companies, including SpaceX and Tesla Inc. Additionally, he has co-founded Neuralink and founded The Boring Company. Musk's innovations and entrepreneurial spirit have been recognized in various fields, including space explorati

In [25]:
##Method 3: Through Contextual models / embedding based models



model_mpnet = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')





modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
def calculate_embedding_similarity(text1, text2):
    # Get embeddings for both texts
    embeddings = model_mpnet.encode([text1, text2])
    # Compute cosine similarity
    similarity = cosine_similarity([embeddings[0]], [embeddings[1]])
    return similarity[0][0]

def QABot(question, llm_chain):
    rag_chain = (
     {"context": db, "question": RunnablePassthrough()}
        | llm_chain
    )
    result = rag_chain.invoke(question)

    context = "".join(str(x) for x in result['context'])
    text = "".join(str(x) for x in result['text'])

    # Compute embedding similarity
    embedding_similarity = calculate_embedding_similarity(context, text)
    print(f"Embedding similarity: {embedding_similarity:.2f}")

    if(embedding_similarity > 0.5):
        print(text)
    else:
        print("Question is out of context")



In [30]:
question = 'What is LUMBAR DISK DISEASE'
QABot(question,llm_chain)

Embedding similarity: 0.66

Lumbar disk disease refers to a condition where the gel-like cushion (disk) between the vertebrae in the lower back becomes damaged or worn out, causing pressure on the surrounding nerves and resulting in pain, numbness, tingling, and weakness in the back and legs. This can lead to a variety of symptoms such as sciatica, radiculopathy, and myelopathy. Diagnosis is typically made through a combination of physical examination, medical history, and imaging tests such as X-rays, MRI, or CT scans. Treatment options include medication, physical therapy, steroid injections, and surgery.


In [31]:
question = 'Who is Elon Musk?'
QABot(question,llm_chain)

Embedding similarity: 0.02
Question is out of context
